In [35]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [37]:
# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 4
EPOCHS = 100
NUM_CLASSES = 8
DATASET_PATH = '/Users/malhar.inamdar/Desktop/road/DATASET_PATH'

In [36]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # 20% validation split
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [38]:
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 201 images belonging to 8 classes.
Found 48 images belonging to 8 classes.


In [30]:
# Compute Class Weights for Imbalanced Data
classes = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(classes),
    y=classes
)
class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)

Class Weights: {0: 1.046875, 1: 1.046875, 2: 1.0923913043478262, 3: 1.0923913043478262, 4: 0.8663793103448276, 5: 1.005, 6: 0.8663793103448276, 7: 1.046875}


In [39]:
# Load Pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [40]:
# Freeze Initial Layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Build Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(0.01))
])

In [42]:
# Compile Model

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
#early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


In [43]:
# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)

# Save Model
model.save('road_construction_stage_classifier_new.h5')

# Evaluate Model
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 41s 674ms/step - accuracy: 0.1255 - loss: 8.2051 - val_accuracy: 0.1458 - val_loss: 6.6941
Epoch 2/100
 1/50 ━━━━━━━━━━━━━━━━━━━━ 13s 272ms/step - accuracy: 0.0000e+00 - loss: 4.5473

2024-11-17 17:56:29.835360: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-17 17:56:29.870535: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


AttributeError: 'NoneType' object has no attribute 'items'

In [26]:
print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", val_generator.samples)
print("Classes:", train_generator.class_indices)
print("Class Weights Dict:", class_weights_dict)


Number of training samples: 201
Number of validation samples: 48
Classes: {'Base_Layer_Construction': 0, 'Clearing_and_Grubbing': 1, 'Excavation_and_Earthwork': 2, 'Macadamisation': 3, 'Pedestrian_Infrastructure': 4, 'Road_Marking': 5, 'Subgrade_Preparation': 6, 'Utility_Installation': 7}
Class Weights Dict: {0: 1.046875, 1: 1.046875, 2: 1.0923913043478262, 3: 1.0923913043478262, 4: 0.8663793103448276, 5: 1.005, 6: 0.8663793103448276, 7: 1.046875}
